In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
from bayes_opt import BayesianOptimization
sns.set()
from pprint import pprint
import json

In [2]:
def get_state(data, t, n):
    d = t - n + 1
    block = data[d : t + 1] if d >= 0 else -d * [data[0]] + data[0 : t + 1]
    res = []
    for i in range(n - 1):
        res.append(block[i + 1] - block[i])
    return np.array([res])

In [3]:
df = pd.read_csv('../dataset/GOOG.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-10-02,959.979980,962.539978,947.840027,953.270020,953.270020,1283400
1,2017-10-03,954.000000,958.000000,949.140015,957.789978,957.789978,888300
2,2017-10-04,957.000000,960.390015,950.690002,951.679993,951.679993,952400
3,2017-10-05,955.489990,970.909973,955.179993,969.960022,969.960022,1213800
4,2017-10-06,966.700012,979.460022,963.359985,978.890015,978.890015,1173900


In [4]:
close = df.Close.values.tolist()
window_size = 30
skip = 5
l = len(close) - 1

In [5]:
class Deep_Evolution_Strategy:

    inputs = None

    def __init__(
        self, weights, reward_function, population_size, sigma, learning_rate
    ):
        self.weights = weights
        self.reward_function = reward_function
        self.population_size = population_size
        self.sigma = sigma
        self.learning_rate = learning_rate

    def _get_weight_from_population(self, weights, population):
        weights_population = []
        for index, i in enumerate(population):
            jittered = self.sigma * i
            weights_population.append(weights[index] + jittered)
        return weights_population

    def get_weights(self):
        return self.weights

    def train(self, epoch = 100, print_every = 1):
        lasttime = time.time()
        for i in range(epoch):
            population = []
            rewards = np.zeros(self.population_size)
            for k in range(self.population_size):
                x = []
                for w in self.weights:
                    x.append(np.random.randn(*w.shape))
                population.append(x)
            for k in range(self.population_size):
                weights_population = self._get_weight_from_population(
                    self.weights, population[k]
                )
                rewards[k] = self.reward_function(weights_population)
            rewards = (rewards - np.mean(rewards)) / np.std(rewards)
            for index, w in enumerate(self.weights):
                A = np.array([p[index] for p in population])
                self.weights[index] = (
                    w
                    + self.learning_rate
                    / (self.population_size * self.sigma)
                    * np.dot(A.T, rewards).T
                )
            if (i + 1) % print_every == 0:
                print(
                    'iter %d. reward: %f'
                    % (i + 1, self.reward_function(self.weights))
                )
        print('time taken to train:', time.time() - lasttime, 'seconds')


class Model:
    def __init__(self, input_size, layer_size, output_size):
        self.weights = [
            np.random.randn(input_size, layer_size),
            np.random.randn(layer_size, output_size),
            np.random.randn(layer_size, 1),
            np.random.randn(1, layer_size),
        ]

    def predict(self, inputs):
        feed = np.dot(inputs, self.weights[0]) + self.weights[-1]
        decision = np.dot(feed, self.weights[1])
        buy = np.dot(feed, self.weights[2])
        return decision, buy

    def get_weights(self):
        return self.weights

    def set_weights(self, weights):
        self.weights = weights

In [6]:
class Agent:
    def __init__(
        self,
        population_size,
        sigma,
        learning_rate,
        model,
        money,
        max_buy,
        max_sell,
        skip,
        window_size,
    ):
        self.window_size = window_size
        self.skip = skip
        self.POPULATION_SIZE = population_size
        self.SIGMA = sigma
        self.LEARNING_RATE = learning_rate
        self.model = model
        self.initial_money = money
        self.max_buy = max_buy
        self.max_sell = max_sell
        self.es = Deep_Evolution_Strategy(
            self.model.get_weights(),
            self.get_reward,
            self.POPULATION_SIZE,
            self.SIGMA,
            self.LEARNING_RATE,
        )

    def act(self, sequence):
        decision, buy = self.model.predict(np.array(sequence))
        return np.argmax(decision[0]), int(buy[0])

    def get_reward(self, weights):
        initial_money = self.initial_money
        starting_money = initial_money
        self.model.weights = weights
        state = get_state(close, 0, self.window_size + 1)
        inventory = []
        quantity = 0
        for t in range(0, l, self.skip):
            action, buy = self.act(state)
            next_state = get_state(close, t + 1, self.window_size + 1)
            if action == 1 and initial_money >= close[t]:
                if buy < 0:
                    buy = 1
                if buy > self.max_buy:
                    buy_units = self.max_buy
                else:
                    buy_units = buy
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
            elif action == 2 and len(inventory) > 0:
                if quantity > self.max_sell:
                    sell_units = self.max_sell
                else:
                    sell_units = quantity
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell

            state = next_state
        return ((initial_money - starting_money) / starting_money) * 100

    def fit(self, iterations, checkpoint):
        self.es.train(iterations, print_every = checkpoint)

    def buy(self):
        initial_money = self.initial_money
        state = get_state(close, 0, self.window_size + 1)
        starting_money = initial_money
        states_sell = []
        states_buy = []
        inventory = []
        quantity = 0
        for t in range(0, l, self.skip):
            action, buy = self.act(state)
            next_state = get_state(close, t + 1, self.window_size + 1)
            if action == 1 and initial_money >= close[t]:
                if buy < 0:
                    buy = 1
                if buy > self.max_buy:
                    buy_units = self.max_buy
                else:
                    buy_units = buy
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
                states_buy.append(t)
                print(
                    'day %d: buy %d units at price %f, total balance %f'
                    % (t, buy_units, total_buy, initial_money)
                )
            elif action == 2 and len(inventory) > 0:
                bought_price = inventory.pop(0)
                if quantity > self.max_sell:
                    sell_units = self.max_sell
                else:
                    sell_units = quantity
                if sell_units < 1:
                    continue
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell
                states_sell.append(t)
                try:
                    invest = ((total_sell - bought_price) / bought_price) * 100
                except:
                    invest = 0
                print(
                    'day %d, sell %d units at price %f, investment %f %%, total balance %f,'
                    % (t, sell_units, total_sell, invest, initial_money)
                )
            state = next_state

        invest = ((initial_money - starting_money) / starting_money) * 100
        print(
            '\ntotal gained %f, total investment %f %%'
            % (initial_money - starting_money, invest)
        )
        plt.figure(figsize = (20, 10))
        plt.plot(close, label = 'true close', c = 'g')
        plt.plot(
            close, 'X', label = 'predict buy', markevery = states_buy, c = 'b'
        )
        plt.plot(
            close, 'o', label = 'predict sell', markevery = states_sell, c = 'r'
        )
        plt.legend()
        plt.show()

In [7]:
def best_agent(
    window_size, skip, population_size, sigma, learning_rate, size_network
):
    model = Model(window_size, size_network, 3)
    agent = Agent(
        population_size,
        sigma,
        learning_rate,
        model,
        10000,
        5,
        5,
        skip,
        window_size,
    )
    try:
        agent.fit(10, 1000)
        return agent.es.reward_function(agent.es.weights)
    except:
        return 0

In [8]:
def find_best_agent(
    window_size, skip, population_size, sigma, learning_rate, size_network
):
    global accbest
    param = {
        'window_size': int(np.around(window_size)),
        'skip': int(np.around(skip)),
        'population_size': int(np.around(population_size)),
        'sigma': max(min(sigma, 1), 0.0001),
        'learning_rate': max(min(learning_rate, 0.5), 0.000001),
        'size_network': int(np.around(size_network)),
    }
    print('\nSearch parameters %s' % (param))
    investment = best_agent(**param)
    print('stop after 100 iteration with investment %f' % (investment))
    if investment > accbest:
        costbest = investment
    return investment

In [9]:
accbest = 0.0
NN_BAYESIAN = BayesianOptimization(
    find_best_agent,
    {
        'window_size': (2, 50),
        'skip': (1, 15),
        'population_size': (1, 50),
        'sigma': (0.01, 0.99),
        'learning_rate': (0.000001, 0.49),
        'size_network': (10, 1000),
    },
)
NN_BAYESIAN.maximize(init_points = 30, n_iter = 50, acq = 'ei', xi = 0.0)

Initialization
----------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   learning_rate |   population_size |     sigma |   size_network |      skip |   window_size | 

Search parameters {'window_size': 15, 'skip': 14, 'population_size': 50, 'sigma': 0.3418950006097839, 'learning_rate': 0.04407103978320809, 'size_network': 136}
time taken to train: 0.08055615425109863 seconds
stop after 100 iteration with investment 0.000000
    1 | 00m00s |    0.00000 |          0.0441 |           49.9614 |    0.3419 |       135.9981 |   14.0621 |       14.9159 | 

Search parameters {'window_size': 4, 'skip': 4, 'population_size': 43, 'sigma': 0.1544696788757519, 'learning_rate': 0.04593637490241277, 'size_network': 191}
time taken to train: 0.06574368476867676 seconds
stop after 100 iteration with investment -9.705400
    2 | 00m00s |   -9.70540 |          0.0459 |           42.5713 |    0.1545 |       1

/home/tensortom/.local/share/virtualenvs/agent-MVd_fBYN/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


stop after 100 iteration with investment 0.000000
    4 | 00m00s |    0.00000 |          0.0395 |           18.4833 |    0.0516 |       784.3283 |   13.1421 |       43.8238 | 

Search parameters {'window_size': 40, 'skip': 13, 'population_size': 21, 'sigma': 0.26353186746777296, 'learning_rate': 0.0788801931587418, 'size_network': 538}
stop after 100 iteration with investment 0.000000
    5 | 00m00s |    0.00000 |          0.0789 |           21.1685 |    0.2635 |       537.7389 |   13.4438 |       39.5727 | 

Search parameters {'window_size': 46, 'skip': 15, 'population_size': 43, 'sigma': 0.6933458151109948, 'learning_rate': 0.4808186411963526, 'size_network': 508}
stop after 100 iteration with investment 0.000000
    6 | 00m00s |    0.00000 |          0.4808 |           42.6508 |    0.6933 |       508.2450 |   14.9141 |       45.6836 | 

Search parameters {'window_size': 14, 'skip': 7, 'population_size': 28, 'sigma': 0.37004153208157065, 'learning_rate': 0.07616716471128422, 'size_ne

time taken to train: 0.7089459896087646 seconds
stop after 100 iteration with investment 2.970298
   27 | 00m00s |    2.97030 |          0.3706 |           41.6542 |    0.2816 |       455.7366 |    4.5550 |       43.3324 | 

Search parameters {'window_size': 46, 'skip': 12, 'population_size': 2, 'sigma': 0.9788476729330938, 'learning_rate': 0.00019375307591891922, 'size_network': 396}
stop after 100 iteration with investment 0.000000
   28 | 00m00s |    0.00000 |          0.0002 |            2.1578 |    0.9788 |       396.4372 |   11.7259 |       45.7265 | 

Search parameters {'window_size': 47, 'skip': 9, 'population_size': 30, 'sigma': 0.7706972021758618, 'learning_rate': 0.415656558008011, 'size_network': 716}
time taken to train: 0.69107985496521 seconds
stop after 100 iteration with investment 0.000000
   29 | 00m00s |    0.00000 |          0.4157 |           30.3610 |    0.7707 |       716.2709 |    9.3417 |       47.1898 | 

Search parameters {'window_size': 36, 'skip': 3, 'popu

/home/tensortom/.local/share/virtualenvs/agent-MVd_fBYN/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


...

/home/tensortom/.local/share/virtualenvs/agent-MVd_fBYN/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide



Search parameters {'window_size': 5, 'skip': 9, 'population_size': 27, 'sigma': 0.736629932616964, 'learning_rate': 0.3024823066148386, 'size_network': 320}
time taken to train: 0.04157829284667969 seconds
stop after 100 iteration with investment 0.000000
   77 | 00m01s |    0.00000 |          0.3025 |           26.6039 |    0.7366 |       320.3396 |    8.7092 |        5.2520 | 

Search parameters {'window_size': 40, 'skip': 7, 'population_size': 30, 'sigma': 0.7900040379815461, 'learning_rate': 0.1771015923003528, 'size_network': 260}
time taken to train: 0.24851465225219727 seconds
stop after 100 iteration with investment 0.000000
   78 | 00m01s |    0.00000 |          0.1771 |           29.8811 |    0.7900 |       259.9906 |    7.4029 |       39.8241 | 

Search parameters {'window_size': 24, 'skip': 4, 'population_size': 4, 'sigma': 0.1805618759961537, 'learning_rate': 0.38108165473366873, 'size_network': 994}
stop after 100 iteration with investment 0.000000
   79 | 00m01s |    0.

/home/tensortom/.local/share/virtualenvs/agent-MVd_fBYN/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide



Search parameters {'window_size': 15, 'skip': 15, 'population_size': 2, 'sigma': 0.5979653657474536, 'learning_rate': 0.19592618408670315, 'size_network': 518}
stop after 100 iteration with investment 0.000000
   80 | 00m01s |    0.00000 |          0.1959 |            2.1262 |    0.5980 |       517.5779 |   14.5137 |       15.1907 | 


/home/tensortom/.local/share/virtualenvs/agent-MVd_fBYN/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


In [10]:
print('Best AGENT accuracy value: %f' % NN_BAYESIAN.res['max']['max_val'])
print('Best AGENT parameters: ', NN_BAYESIAN.res['max']['max_params'])

Best AGENT accuracy value: 2.970298
Best AGENT parameters:  {'window_size': 43.33243057107037, 'skip': 4.55503297305148, 'population_size': 41.65417279935052, 'sigma': 0.28159266059192467, 'learning_rate': 0.3705986604554016, 'size_network': 455.7366319925798}


In [11]:
bayes_res = str(NN_BAYESIAN.res['max']['max_params'])
bayes_res = json.loads(bayes_res.replace("'", "\""))
bayes_res['window_size'] = int(round(bayes_res['window_size']))
bayes_res['skip'] = int(round(bayes_res['skip']))
bayes_res['population_size'] = int(round(bayes_res['population_size']))
bayes_res['size_network'] = int(round(bayes_res['size_network']))
pprint(bayes_res)

{'learning_rate': 0.3705986604554016,
 'population_size': 42,
 'sigma': 0.28159266059192467,
 'size_network': 456,
 'skip': 5,
 'window_size': 43}


In [12]:
# this is my parameter
best_agent(30, 1, 15, 0.1, 0.03, 500)

time taken to train: 0.7401294708251953 seconds


2.4318994100000237

In [13]:
# this is from bayesian
best_agent(bayes_res['window_size'], bayes_res['skip'], bayes_res['population_size'], bayes_res['sigma'], bayes_res['learning_rate'], bayes_res['size_network'])

time taken to train: 0.6380605697631836 seconds


0.8894995000000017

In [15]:
# this is my parameter
model = Model(30, 500, 3)
agent = Agent(15, 0.1, 0.03, model, 10000, 5, 5, 1, 30)
agent.fit(500, 100)
agent.buy()

ValueError: shapes (1,43) and (30,500) not aligned: 43 (dim 1) != 30 (dim 0)

In [ ]:
# this is from bayesian
model = Model(20, 176, 3)
#agent = Agent(bayes_res['population_size'], bayes_res['sigma'], bayes_res['learning_rate'], model, 10000, 5, 5, bayes_res['skip'], bayes_res['window_size'])
agent = Agent(
    34, 0.7152490670931403, 0.28208006704411104, model, 10000, 5, 5, 1, 20
)
agent.fit(500, 100)
agent.buy()